# Alexandria glass recycling locations relative to where people live.

More people will drop off glass and be happy with the program if the drop off locations are convenient.  An easy measure of convenience is proximity to residential neighborhoods.  Another useful benchmark is proximity to facilities that the city tried to centrally locate, such as libraries and fire stations.  Proximity to other facilities that people visit for  other reasons might also be helpful and we could add them to a future map.

This graph plots one mile rings around the City of Alexandria, Virginia glass recycling locations while plotting Census Block Group level population density.  [The data are 2022 American Community Survey (ACS) 5 year estimates published on CensusReporter.org](https://censusreporter.org/data/table/?table=B01003&geo_ids=16000US5101000,150|16000US5101000&primary_geo_id=16000US5101000).  I downloaded the data in GeoJSON format.  

I hope that this work is useful both for advancing understanding of Alexandria's glass recycling program and for advancing local government policy discussions.

Some of this program is [based on this example code](https://vverde.github.io/blob/interactivechoropleth.html).  I also referenced [this example ](https://github.com/yohman/getting-started-with-gis/blob/master/Intro%20to%20GIS.ipynb)

I have called Alexandria home for 15 years.  I do policy analysis and data work professionally.  I hope this is a useful starting place for conversation. I am doing this in my personal capacity as a citizen and the work here is not endorsed or approved by my employer.

Rob L.
You can contact me by opening an issue on this project or by <a href="mailto:RobertUtilityMan@gmail.com">e-mail</a>




In [ ]:
#import necessary Python geographic data analysis and interactive mapping libraries
import geopandas as gpd
import folium  #folium is a Python interface for creating leaflet.js maps
import numpy as np
from shapely.geometry.polygon import orient


: 

### Load and examine the data.

In [ ]:
geojson_filename = 'acs2022_5yr_B01003_15000US515102013004.geojson'

alx = gpd.read_file(geojson_filename)
#give the population column a more descriptive name
alx.rename(columns={'B01003001':'population'}, inplace=True)

alx.head()

: 

In [ ]:
alx.columns

: 

## Calculate the area of each block group, so we can compute population density

Calculating areas is tricky because coordinates are on Earth, which is a spheroid, but need to be projected onto a plane to make area estimation tractable and, sadly, the Python tools have not settled on a single, obvious approach that just works and hides these complexities from the programmer.  The obvious GeoPandas geodf['geometry'].area function yielded puzzling answers.  I drew on this [sample code and discussion](https://gis.stackexchange.com/questions/413349/calculating-area-of-lat-lon-polygons-without-transformation-using-geopandas) to produce valid answers. 


In [ ]:
def gpd_geographic_area(geodf):
    if not geodf.crs and geodf.crs.is_geographic:
        raise TypeError('geodataframe should have geographic coordinate system')
        
    geod = geodf.crs.get_geod()
    def area_calc(geom):
        if geom.geom_type not in ['MultiPolygon','Polygon']:
            return np.nan
        
        # For MultiPolygon do each separately
        if geom.geom_type=='MultiPolygon':
            return np.sum([area_calc(p) for p in geom.geoms])

        # orient to ensure a counter-clockwise traversal. 
        # See https://pyproj4.github.io/pyproj/stable/api/geod.html
        # geometry_area_perimeter returns (area, perimeter)
        return geod.geometry_area_perimeter(orient(geom, 1))[0]
    
    return geodf.geometry.apply(area_calc)
#compute the physical size of each block group

alx['area'] = gpd_geographic_area(alx)/2589988.10 #divide by the number of square meters per square miles

#compute the population density per square mile of each block group
alx['pop_density'] = alx['population'] / alx['area']



: 

## Sanity check the population density against published figures.  

Look at Alexandria as a whole and compare to the published figures that Alexandria has a 2021 population of 154,706 spread over 15.35 square miles.  The population density below of about 10,000 residents per square mile is consistent with those numbers.

In [ ]:
alx[['name','area','population','pop_density']][-1:]

: 

## Look at the end of the data frame, where the city wide rows used to be.

In [ ]:
alx.tail()

: 

## Exclude city-wide rows, so we map just block groups.

In [ ]:
#print sizes before and after the subsetting operation to make sure it worked
print(len(alx))
alx=alx[alx["population"]<157594.0].copy()
print(len(alx))

: 

## Look at the end of the data frame, where the city wide rows used to be.

In [ ]:
alx.tail()

: 

## Create a map centered on Alexandria.

In [ ]:
#center the map on the average centroid X and Y coordinate of the block groups.  This emulates an example and works well enough.
longitude=alx.centroid.x.mean()
latitude=alx.centroid.y.mean()
print(longitude,latitude)
alx_glass_recycling_map = folium.Map(location=[latitude, longitude], zoom_start=13.48)  



: 

## Compute the maximum population density in the city

This allows us to set up an appropriate scale of colors that starts light for low density and makes the highest density block groups dark

In [ ]:
alx['pop_density'].max()

: 

## Create the color scale.  

There are a few very dense block groups, for example, at the Southern Towers.  Most blocks groups are much less dense.  There's no value in distinguishing between a population density of 60,000-64,999 people per square mile and a density of 65,000-69,999 because there are no block groups in the former category and both are among the densest in the city.  There is a lot of value to distinguishing between a density of 10,000-14,9999 and 15,000-19,999 because there are many of each and the latter are often noticeably denser.  Thus, adding more gradations at lower density levels seems reasonable.

In [ ]:
#remember that range stops one bin before you expect it to, so the top of the bin range is actually 75,000
#split bins every 5000 people per square mile up to 30,000 people per square mile and then every 15,000 people per square mile after that
myscale = list(range(0, 35000, 5000))
myscale+=list(range(30000, 90000, 15000))
myscale

: 

In [ ]:
#create a copy of the 'pop_density' column, round it to 1 decimal place, apply a comma separator, and convert it to a string
alx['population density per sq. mile']= alx['pop_density'].apply(lambda x: "{:,.1f}".format(x)).astype(str)

#add the choropleth layer showing block groups and their population density
choropleth = folium.Choropleth(
    geo_data=alx,
    name='Population density',
    data=alx,
    columns=['geoid','pop_density'],
    fill_color='YlGnBu',
    threshold_scale=myscale,
    fill_opacity=.5,
    line_opacity=0.2,
    legend_name='Population density',
    smooth_factor=0,
    key_on="feature.properties.geoid",  # Uncomment this line if your geojson contains 'geoid' property
    control=False
)


#add mouse over text to alx_glass_recycling_map
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name','population', 'population density per sq. mile'], 
                                   localize=True, # here, localization turns on comma separators on the population variable.  Pop density per square mile is already a string formatted with commas,
                                   #control=False
                                   )
)

# we need to add here to get the combined element that consists of folium.Choropleth and the GeoJsonTooltip
# to respect the name and control parameters in folium.Choropleth
# if we ....add_child(...).add_to(alx_glass_recycling_map), the name and control properties will not be inherited from either source
choropleth.add_to(alx_glass_recycling_map)


: 

# Confirm that the steps so far have yielded a map that color codes the city by population density

In [ ]:
alx_glass_recycling_map

: 

# Add 1 mile radius circles to the map at the locations of each of the Alexandria Glass Recycling Drop Off locations.

In [ ]:
location_list = [
    { "coordinates": [38.841095,-77.061798],                  "description":"MOM's Organic Market Recycling Drop Off, 3831 Mount Vernon Avenue"},
    { "coordinates": [38.806963637752006, -77.0863824995479], "description":"3224 Colvin Street"},
    { "coordinates": [38.81440032023959, -77.13641565531925], "description":"S. Whiting St. and Tower Court"},
    { "coordinates": [38.80455939216254, -77.10741058474441], "description":"4251 Eisenhower Ave"},
    { "coordinates": [38.794426093278446, -77.04568977469864], "description":"Jones Point Park"},
]

#add a layer of circles to the map at the locations of each of the Alexandria Glass Recycling Drop Off locations

circle_group = folium.FeatureGroup(name='Existing Glass Recycling Locations')

for loc in location_list:
    #804.5 meters is half a mile; 1609 meters is one mile
    folium.Circle(radius=1609,location=loc["coordinates"],color='purple',fill=True,fill_opacity=0.05,popup=loc["description"]).add_to(circle_group)

# Add the group to the map
circle_group.add_to(alx_glass_recycling_map)




: 

# Add small circles to the locations we mention in our query

In [ ]:
locations_discussed_group = folium.FeatureGroup(name='Potential new location')

location_list = [
    { "coordinates": [38.816253, -77.052979], "description":"Option 1:  The last parking spaces on the east side of Main Line Boulevard"},
    { "coordinates": [38.821129, -77.053557], "description":"Option 2:  Eugene Simpson Park"},
]


for loc in location_list:
    #CircleMarker has a radius specified in pixels, while Circle is specified in meters. 
    folium.CircleMarker(radius=5,location=loc["coordinates"],color='purple',fill=True,fill_opacity=1,popup=loc["description"]).add_to(locations_discussed_group)

# Add the group to the map
locations_discussed_group.add_to(alx_glass_recycling_map)


: 

# Add city library locations

In [ ]:
library_group = folium.FeatureGroup(name='Public Libraries')

location_list = [
  {"branch":"Barrett Branch Library", "coordinates": [38.807991762388724, -77.04713524593498  ]
  },
  {"branch":"Beatley Central Library", "coordinates":[    38.8122619360386, -77.11731906006739]
  },
  {"branch":"Duncan Branch Library", 
   "coordinates": [    38.83505605577527, -77.06116591995838]
  },
  {"branch":"Burke Branch Library", 
   "coordinates": [   38.82787,-77.1100971]
  }
]


for loc in location_list:
    #CircleMarker has a radius specified in pixels, while Circle is specified in meters. 
    folium.RegularPolygonMarker(radius=3,number_of_sides=4, location=loc["coordinates"],color='blue',fill=True,fill_opacity=1,popup=loc["branch"]).add_to(library_group)

# Add the group to the map
library_group.add_to(alx_glass_recycling_map)

: 

# Add fire stations

In [ ]:
fire_station_group = folium.FeatureGroup(name='Fire Stations')

location_list = [
  {"branch":"Fire Station 201", "coordinates": [38.803688559431485, -77.043336217064]
  },
  {"branch":"Fire Station 202", "coordinates": [38.82480345100984, -77.05728739007715]  },
  {"branch":"Fire Station 203", "coordinates": [38.83020056029216, -77.07386406918509]  },
  {"branch":"Fire Station 204", "coordinates": [38.816571755518304, -77.04669644774889]  },
  {"branch":"Fire Station 205", "coordinates": [38.80704241025861, -77.05277753240662]  },
  {"branch":"Fire Station 206", "coordinates": [38.82676195427494, -77.10888582261215]  },
  
  {"branch":"Fire Station 207", "coordinates": [38.808656635914204, -77.08704687103582]  },
  {"branch":"Fire Station 208", "coordinates": [38.81414958477803, -77.12331301891336]  },
  {"branch":"Fire Station 209", "coordinates": [38.83127326893022, -77.05037833461546]  },
  {"branch":"Fire Station 210", "coordinates": [38.801590558622806, -77.1267041434138]  },
]


for loc in location_list:
    #CircleMarker has a radius specified in pixels, while Circle is specified in meters. 
    folium.RegularPolygonMarker(radius=3,number_of_sides=3, location=loc["coordinates"],color='red',fill=True,fill_opacity=1,popup=loc["branch"]).add_to(fire_station_group)

# Add the group to the map
fire_station_group.add_to(alx_glass_recycling_map)

: 

In [ ]:
#add a control that will let users show or hide the layers and make the options visible by default
#there is commentary about how to add a richer legend here:  https://stackoverflow.com/questions/37466683/create-a-legend-on-a-folium-map
# and here:  https://nbviewer.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd
folium.LayerControl(collapsed=False).add_to(alx_glass_recycling_map)



: 

In [ ]:
alx_glass_recycling_map

: 

# Export the folium map to an HTML file

In [ ]:
alx_glass_recycling_map.save(r'D:\rjl\docs\glass_recycling\alexandria_glass_recycling_raw.html')

: 

# Add custom text to the HTML

In [ ]:
with open(r'D:\rjl\docs\glass_recycling\alexandria_glass_recycling_raw.html', "r") as html_input_file:
    with open(r'D:\rjl\docs\glass_recycling\index.html', "w") as html_output_file:
        html_text = html_input_file.read()
        html_text = html_text.replace("<body>", """<body>
        <H1>Alexandria Glass Recycling Locations and Population Density</H1>
    <P>Alexandria citizens created this website in support of a <a href="https://alx-glass.github.io/query">query</a> to city staff and leaders about potential improvements to glass recycling in the city we are proud to call home.  It shows population density by census block group for the city of Alexandria and one mile rings around each of the city's five glass recycling drop offs.  Blue diamonds show library branches and red triangles show fire stations.  The city has ten fire stations and five glass drop offs.  Four of the ten fire stations are more than a mile from a glass drop off.  Much of the city is far from any drop off location.</p>
    
    <P>We made it using American Community Survey data and free tools as documented in <a href="https://github.com/alx-glass/alx-glass.github.io/blob/main/alexandria_glass_recycling.ipynb">this code</a>. We would be thrilled if citizens learned from our work to illustrate the effects of public programs and inform dialog in their own communities.  We welcome questions and comments by <a href="mailto:RobertUtilityMan@gmail.com">e-mail</a>.</p>""")
        html_output_file.write(html_text)



: 